In [1]:
def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are awesome"

def greet_bob(greeter_func):
    return greeter_func("Bob")

In [2]:
def parent(num):
    def first_child():
        return "Hi I am Emma"
    def second_child():
        return "Call me Liam"
    
    if num == 1:
        return first_child
    else:
        return second_child

In [3]:
# SECTION simple decorators

def my_decorator(func):
    def wrapper():
        print("before func")
        func()
        print("after func")
    return wrapper

def say_whee2():
    print("whee")

say_whee2 = my_decorator(say_whee2)

In [4]:
# SECTION reusing decorators
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs): #IMPT return a function that accepts arguements!
        func(*args, **kwargs)
        return func(*args, **kwargs) # IMPT this will allow the decorated function to continue returning values
    return wrapper_do_twice

# call this from decorators2.py! SECTION go to decorators2.py

In [5]:
if __name__ == "__main__":
    print(greet_bob(say_hello)) # Hello Bob
    print(greet_bob(be_awesome)) # Yo Bob, together we are awesome

    print(parent(1)) # <function parent.<locals>.first_child at 0x7f885657a7a0>
    print(parent(2)) # <function parent.<locals>.second_child at 0x7f885657a830>
    print(parent(1)())
    print(parent(2)())

    print(say_whee2()) # IMPT prints None at the end because say_whee2 is now wrapper which returns nothing
    print(say_whee2) # IMPT <function my_decorator.<locals>.wrapper at 0x7fb323e668c0>

Hello Bob
Yo Bob, together we are awesome
<function parent.<locals>.first_child at 0x7f8525e7ef20>
<function parent.<locals>.second_child at 0x7f8525e7efc0>
Hi I am Emma
Call me Liam
before func
whee
after func
None
<function my_decorator.<locals>.wrapper at 0x7f8525e7ec00>


In [14]:
# SECTION continuation from 1decorators

@do_twice
def say_whee():
    print("whee")

@do_twice
def greet(name):
    print(f"hello {name}")

In [16]:
# SECTION returning values from decorated functions
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}" #IMPT this return statement means the wrapper in the decorator must return the original function

In [18]:
if __name__ == "__main__":
    say_whee()
    greet("world") # IMPT this works because wrapper_do_twice that was returned accepts args and kwargs!
    hi_adam = return_greeting("Adam")
    print(hi_adam)

    #SECTION checking on __name__ and using ...
    print
    print.__name__
    help(print) # press 'q' to continue
    # IMPT we can use this for defined functions
    say_whee
    say_whee.__name__
    help(say_whee) # IMPT Help on function wrapper_do_twice in module decorators1:wrapper_do_twice(*args, **kwargs)
    #IMPT we will see how we can change the name back in decorators3!

whee
whee
hello world
hello world
Creating greeting
Creating greeting
Hi Adam
Help on built-in function print in module builtins:

print(*args, sep=' ', end='\n', file=None, flush=False)
    Prints the values to a stream, or to sys.stdout by default.
    
    sep
      string inserted between values, default a space.
    end
      string appended after the last value, default a newline.
    file
      a file-like object (stream); defaults to the current sys.stdout.
    flush
      whether to forcibly flush the stream.

Help on function wrapper_do_twice in module decorators1:

wrapper_do_twice(*args, **kwargs)



In [20]:
import functools
import time

# SECTION make say_whee() to still be itself after decoration

def do_twice(func):
    @functools.wraps(func) # IMPT 
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def say_whee():
    print("whee")

In [21]:
# SECTION IMPT a good boilerplate:
def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # do something
        value = func(*args, **kwargs)
        # do something
        return value
    return wrapper_decorator

In [23]:
# SECTION timer decorator
def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [24]:
if __name__ == "__main__":
    say_whee
    say_whee.__name__
    help(say_whee)
    say_whee()
    waste_some_time(1)
    waste_some_time(999)

Help on function say_whee in module __main__:

say_whee()

whee
whee
Finished 'waste_some_time' in 0.0004 secs
Finished 'waste_some_time' in 0.2908 secs


In [25]:
# SECTION debugging code

import functools
import math

def debug(func):
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args] #IMPT list of positional arguments in string
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()] # IMPTTODOlist of keyyword arguments, !r means that repr() is used to represent the value
        signature = ", ".join(args_repr + kwargs_repr) #IMPT created by joining the string represenations of all arguments
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"Calling {func.__name__!r} returned {value!r}")
        return value
    return wrapper_debug

@debug
def make_greeting(name, age = None):
    if age is None:
        return f"Howdy {name}!"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"

In [26]:
# SECTION applying debug to math function
math.factorial = debug(math.factorial) # same as using decorator!

def approximate_e(terms = 18):
    print(sum(1 / math.factorial(n) for n in range(terms)))

In [27]:
if __name__ == "__main__":
    make_greeting("Benjamin")
    make_greeting("Richard", age=112)
    make_greeting(name="Dorrisile", age=116)

    # IMPT this will print the factorial function 5 times!
    approximate_e(5)

Calling make_greeting('Benjamin')
Calling 'make_greeting' returned 'Howdy Benjamin!'
Calling make_greeting('Richard', age=112)
Calling 'make_greeting' returned 'Whoa Richard! 112 already, you are growing up!'
Calling make_greeting(name='Dorrisile', age=116)
Calling 'make_greeting' returned 'Whoa Dorrisile! 116 already, you are growing up!'
Calling factorial(0)
Calling 'factorial' returned 1
Calling factorial(1)
Calling 'factorial' returned 1
Calling factorial(2)
Calling 'factorial' returned 2
Calling factorial(3)
Calling 'factorial' returned 6
Calling factorial(4)
Calling 'factorial' returned 24
2.708333333333333


In [28]:
# SECTION SLOWING DOWN CODE
import functools
import time
def slow_down(func):
    """Sleep 1 second before calling the function"""
    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down

@slow_down
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)

#### Plugins: Decorators don’t have to wrap the function they’re decorating. They can also simply register that a function exists and return it unwrapped.

#### note the syntax of random.choice(list(PLUGINS.items()))

.items() returns a view object with pairs as tuples ie. dict_items([(k, v), (k, v)]). using list() converts the view object into a list [(k, v), (k, v)]

In [39]:
# SECTION register plugins

import random
PLUGINS = dict()

def register(func):
    """Register a function as a plug-in"""
    PLUGINS[func.__name__] = func
    return func # just returns the original function

@register
def say_hello(name):
    return f"Hello {name}"

@register
def be_awesome(name):
    return f"Yo {name}, together we are awesome"

def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items())) #IMPT note the syntax choose from the list a key-value to destructure. items() creates a view object
    print(f"Using {greeter!r}")
    return greeter_func(name)

In [44]:
if __name__ == "__main__":
    countdown(3)
    print(randomly_greet("Alice"))
    print(PLUGINS) # IMPT returns the dictionary
    print(globals()) # IMPT this gives access to all global variables, similar to PLUGINS


3
2
1
Liftoff!
Using 'say_hello'
Hello Alice
None
{'say_hello': <function say_hello at 0x7f8503f365c0>, 'be_awesome': <function be_awesome at 0x7f8503f37740>}
{'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'def say_hello(name):\n    return f"Hello {name}"\n\ndef be_awesome(name):\n    return f"Yo {name}, together we are awesome"\n\ndef greet_bob(greeter_func):\n    return greeter_func("Bob")', 'def parent(num):\n    def first_child():\n        return "Hi I am Emma"\n    def second_child():\n        return "Call me Liam"\n    \n    if num == 1:\n        return first_child\n    else:\n        return second_child', '# SECTION simple decorators\n\ndef my_decorator(func):\n    def wrapper():\n        print("before func")\n        func()\n        print("after func")\n    r

In [49]:
#SECTION user logged IMPT should use Flask-login extension instead

from flask import Flask, g, request, redirect, url_for
import functools
app = Flask(__name__)

def login_required(func):
    """Make sure user is logged in before proceeding"""
    @functools.wraps(func)
    def wrapper_login_required(*args, **kwargs):
        if g.user is None:
            return redirect(url_for("login", next=request.url))
        return func(*args, **kwargs)
    return wrapper_login_required

@app.route("/secret")
@login_required
def secret():
    ...

# Fancy decorators

### Decorating classes

Some commonly used decorators that are even built-ins in Python are @classmethod, @staticmethod, and @property. The @classmethod and @staticmethod decorators are used to define methods inside a class namespace that are not connected to a particular instance of that class. The @property decorator is used to customize getters and setters for class attributes.

In [54]:
# SECTION @classmethod, @staticmethod, @property

class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        """Get value of radius"""
        return self._radius
    
    @radius.setter
    def radius(self, value):
        """Set radius, raise error if negative"""
        if value >= 0:
            self._radius = value
        else:
            raise ValueError("Radius must be positive")
        
    @property
    def area(self):
        """Calculate area inside circle"""
        return self.pi() * self.radius**2
    
    def cylinder_volume(self, height):
        """Calculate volume of cylinder with circle as base"""
        return self.area * height
    
    @classmethod # IMPT does not take into account the instance. Can call with or without instance, the class is passed and the instance if passed it DROPPED
    def unit_circle(cls):
        """Factory method creating a circle with radius 1"""
        return cls(1)
    
    @staticmethod # IMPT does not take into account the class AND instance. Can call with or without instance but both class and instance will be DROPPED 
    def pi():
        """Value of pi, could use math.pi though"""
        return 3.1415926535



.unit_circle() is a class method. It’s not bound to one particular instance of Circle. Class methods are often used as factory methods that can create specific instances of the class.

.pi() is a static method. It’s not really dependent on the Circle class, except that it is part of its namespace. Static methods can be called on either an instance or the class.

In [57]:
c = Circle(5)
print(c.radius)
print(c.area)
c.radius = 2
print(c.area)
# c.area = 100 cannot be set!
print(c.cylinder_volume(height = 4))
# c.radius = -1 cannot be set!
c = Circle.unit_circle() # IMPT factory method
print(c.radius)
print(c.pi())
print(Circle.pi())

5
78.5398163375
12.566370614
50.265482456
1
3.1415926535
3.1415926535


1. Custom decorators for classes:

In [65]:
# need to run debug and timer first!

class TimeWaster:
    @debug
    def __init__(self, max_num):
        self.max_num = max_num

    @timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

tw = TimeWaster(1000)
print(tw)
print(tw.waste_time)
print(tw.waste_time(999))

# Calling __init__(<__main__.TimeWaster object at 0x7f850384dcd0>, 1000)
# Calling '__init__' returned None
# <__main__.TimeWaster object at 0x7f850384dcd0>
# <bound method TimeWaster.waste_time of <__main__.TimeWaster object at 0x7f850384dcd0>>
# Finished 'waste_time' in 0.0295 secs
# None

Calling __init__(<__main__.TimeWaster object at 0x7f8503f8e890>, 1000)
Calling '__init__' returned None
<bound method TimeWaster.waste_time of <__main__.TimeWaster object at 0x7f8503f8e890>>
Finished 'waste_time' in 0.0311 secs
None


2. Decorate the whole class

In [66]:
from dataclasses import dataclass

@dataclass
class PlayingCard:
    rank: str
    suit: str

The meaning of the syntax is similar to the function decorators. In the example above, you could have done the decoration by writing PlayingCard = dataclass(PlayingCard).

A common use of class decorators is to be a simpler alternative to some use-cases of metaclasses. In both cases, you are changing the definition of a class dynamically.

Writing a class decorator is very similar to writing a function decorator. The only difference is that the decorator will receive a class and not a function as an argument. In fact, all the decorators you saw above will work as class decorators. When you are using them on a class instead of a function, their effect might not be what you want. In the following example, the @timer decorator is applied to a class:

In [67]:
@timer
class TimeWaster:
    def __init__(self, max_num):
        self.max_num = max_num

    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

Decorating a class does not decorate its methods. Recall that @timer is just shorthand for TimeWaster = timer(TimeWaster).

Here, @timer only measures the time it takes to instantiate the class:

In [68]:
tw = TimeWaster(1000)
tw.waste_time(999)

Finished 'TimeWaster' in 0.0000 secs


## Nesting decorators

In [70]:
@debug
@do_twice
def greet(name):
    print(f"Hello {name}")

greet("Eva")

Calling greet('Eva')
Hello Eva
Hello Eva
Calling 'greet' returned None


@debug calls @do_twice, which calls greet(), or debug(do_twice(greet())):

Swapping them applies @do_twice on @debug as well

In [72]:
@do_twice
@debug
def greet(name):
    print(f"Hello {name}")

greet("Eva")

Calling greet('Eva')
Hello Eva
Calling 'greet' returned None
Calling greet('Eva')
Hello Eva
Calling 'greet' returned None
